# Hledání maximální kliky grafu pomocí branch and bound algoritmu

Co je maximální klika grafu?

**Přilehlé vrcholy**

*Přilehlé vrcholy jsou dva vrcholy, které jsou koncovými body stejné hrany.*

**Klika**

*Klika, $C$, v neorientovaném grafu $G=(V,E)$ je podmnožina vrcholů, $C\subseteq V$ tak, že každé dva odlišné vrcholy jsou přilehlé. Ekviv. je to každý úplný podgraf grafu.*

**Maximální klika**

*Maximální klika grafu, $M$, je klika, taková, že neexistuje žádná klika s více vrcholy.*

V prvním kroku nalezneme spodní a vrchní mez pro maximální kliku. Horní mez nalezneme pomocí greedy heuristiky na minimální obarvení grafu. Spodní mez najdeme pomocí greedy clique heuristiky (Bron-Kerboschův algoritmus). Tím máme hotovou "Bound" část a přecházíme k "Branch" části. Použili jsme následující branching procedůru:


*   Mějme graf $G=(V,E)$ a vybrané branching vrcholy $v\in V$, kde $v$ jsou jakékoliv vrcholy, které nejsou spojené se všei ostatními vrcholy v $G$. Pokud takové $v$ neexistuje, potom $G$ je klika.
*   Vytvoříme grafy $G'$ a $G''$ z $G$ takto:

1.   $G'$ je z $G$ vytvořeno jako $V-\{v\}$
2.   $G''$ je z $G$ indukováno vrcholy $v$ z $N(v)$, kde $N(v)$ značí sousedy vrcholu $v$ a $v$. 

Vybereme z grafů $G'$ a $G''$ ten s maximální délkou a řekneme, že se jedná o maximální kliku grafu $G$.




















In [51]:
# requirements
import os
import sys
import threading
from contextlib import contextmanager
import _thread
import time
import networkx as nx # package for graphs and networks
import argparse

In [53]:
# Greedy search algorithm
def greedyHeuristic(graph):
    K = set()
    nodes = [node[0] for node in sorted(nx.degree(graph), key=lambda x: x[1], reverse=True)]
    while len(nodes) != 0:
        neigh = list(graph.neighbors(nodes[0]))
        K.add(nodes[0])
        nodes.remove(nodes[0])
        nodes = list(filter(lambda x: x in neigh, nodes))
    return K

In [54]:
# Greedy graph coloring heuristic
def greedyColoring(graph):
    colorNum = iter(range(0, len(graph)))
    colorMap = {}
    usedColors = set()
    nodes = [node[0] for node in sorted(nx.degree(graph),
                                        key=lambda x: x[1], reverse=True)]
    colorMap[nodes.pop(0)] = next(colorNum)  # color node with color code
    usedColors = {i for i in colorMap.values()}
    while len(nodes) != 0:
        node = nodes.pop(0)
        neighborsColors = {colorMap[neighbor] for neighbor in
                            list(filter(lambda x: x in colorMap, graph.neighbors(node)))}
        if len(neighborsColors) == len(usedColors):
            color = next(colorNum)
            usedColors.add(color)
            colorMap[node] = color
        else:
            colorMap[node] = next(iter(usedColors - neighborsColors))
    return len(usedColors)

In [66]:
# read graph
graph1 = "anna.col"
graph2 = "homer.col"
graph3 = "huck.col"
edges = []
# CHANGE ARGUMENT IN OPEN() FOR DIFFERENT GRAPH
with open(graph1, 'r') as f:
  for lines in f:
    if lines.startswith('p'):
      p, name, vertricesNum, edgesNum = lines.split()
      print("{} {} {}".format(name, vertricesNum, edgesNum))
    elif lines.startswith('e'):
      _, v1, v2 = lines.split()
      vert = (v1,v2)
      edges.append(vert)
    elif lines.startswith('c'):
      print(*lines.split()[1:])
    else:
      continue

graph = nx.Graph(edges)
clique = greedyHeuristic(graph)
chromaticNum = greedyColoring(graph)
if len(clique) == chromaticNum:
    maxCliq = clique
else:
  #branching
  g1 = graph.copy()
  g2 = graph.copy()
  maxNodeDeg = len(graph) -1
  #sort nodes by Deg
  nodesByDeg = [node for node in sorted(nx.degree(graph), key=lambda x: x[1], reverse=True)]
  #nodes witch 'current clique size < Deg < max possible Deg'
  partialConnectedNodes = list(filter(lambda x: x[1] != maxNodeDeg and x[1] <= maxNodeDeg, nodesByDeg))
  g1.remove_node(partialConnectedNodes[0][0])
  # graph without nodes which is not connected with partial connected node with highest degree
  g2.remove_nodes_from(graph.nodes() - graph.neighbors(partialConnectedNodes[0][0]) - {partialConnectedNodes[0][0]})
  def branchBound(graph):
      maxClique = greedyHeuristic(graph)
      chromaticNum = greedyColoring(graph)
      if len(maxClique) == chromaticNum:
          return maxClique
      else:
          g1 = graph.copy()
          g2 = graph.copy()
          maxNodeDeg = len(graph) -1
          nodesByDeg = [node for node in sorted(nx.degree(graph), key=lambda x: x[1], reverse=True)]
          partialConnectedNodes = list(filter(lambda x: x[1] != maxNodeDeg and x[1] <= maxNodeDeg, nodesByDeg))
          g1.remove_node(partialConnectedNodes[0][0])
          g2.remove_nodes_from(graph.nodes() - graph.neighbors(partialConnectedNodes[0][0]) - {partialConnectedNodes[0][0]})
          return max(branchBound(g1), branchBound(g2), key=lambda x: len(x))
  maxCliq = max(branchBound(g1), branchBound(g2), key=lambda x: len(x))
  
print('\nMaximum clique', maxCliq, '\nlength:', len(maxCliq))

FILE: anna.col
Translated from Stanford GraphBase File: anna.gb
Stanford GraphBase ID: book(?anna?,138,0,1,239,0,0,0)
edge 138 986

Maximum clique {'91', '116', '74', '81', '135', '138', '18', '99', '136', '7', '36'} 
length: 11
